In [2]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from war_functions.pecota_tables import *
from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *
from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *
from daily_adjustments.todays_game_info import *
from daily_adjustments.starting_rotations_WAR import *
from daily_adjustments.adjusted_war_today import *
from odds_and_other_projections import *

In [4]:
bets = pd.read_csv('past_bets/base/bets_20220728.csv', index_col = 0)

In [5]:
bets = bets[bets.Home_Team != 'Los Angeles Angels']
bets.reset_index(drop = True, inplace = True)
bets

,Home_Team,Away_Team,Home_Prob,Away_Prob,Home_ML,Away_ML,Home_ML_Prob,Away_ML_Prob,Home_Diff,Away_Diff,Home_KC,Away_KC,Home_Bet,Away_Bet
0,Pittsburgh Pirates,Philadelphia Phillies,0.355337,0.644663,160.0,-190.0,0.384615,0.655172,-0.029279,-0.010509,0.000000,0.000000,0.000000,0.000000
1,New York Yankees,Kansas City Royals,0.717283,0.282717,-235.0,194.0,0.701493,0.340136,0.015791,-0.057419,0.005290,0.000000,594.025598,0.000000
2,Toronto Blue Jays,Detroit Tigers,0.715937,0.284063,-225.0,188.0,0.692308,0.347222,0.023630,-0.063159,0.007680,0.000000,862.377602,0.000000
3,Boston Red Sox,Cleveland Guardians,0.385315,0.614685,100.0,-118.0,0.500000,0.541284,-0.114685,0.073400,0.000000,0.016001,0.000000,1796.856321
4,Houston Astros,Seattle Mariners,0.576172,0.423828,-148.0,126.0,0.596774,0.442478,-0.020602,-0.018650,0.000000,0.000000,0.000000,0.000000
5,Colorado Rockies,Los Angeles Dodgers,0.370594,0.629406,180.0,-215.0,0.357143,0.682540,0.013451,-0.053134,0.002092,0.000000,234.970490,0.000000
6,San Francisco Giants,Chicago Cubs,0.621081,0.378919,-146.0,124.0,0.593496,0.446429,0.027586,-0.067510,0.006786,0.000000,762.035904,0.000000


In [6]:
bets.to_csv('past_bets/base/bets_20220728.csv')

In [3]:
# Parameters
current_year = dt.date.today().year
today = str(dt.date.today()).replace('-', '')
today_date = dt.date.today()
yesterday = today_date - dt.timedelta(days=1)
yesterday_string = str(yesterday)
yesterday_string = yesterday_string.replace('-', '')
kelly = 10
team_map = {
    'Giants' : 'San Francisco Giants',
    'Astros' : 'Houston Astros',
    'Brewers': 'Milwaukee Brewers',
    'Dodgers': 'Los Angeles Dodgers',
    'Rays' : 'Tampa Bay Rays',
    'Red Sox': 'Boston Red Sox',
    'White Sox': 'Chicago White Sox',
    'Padres': 'San Diego Padres',
    'Athletics': 'Oakland Athletics',
    'Yankees' : 'New York Yankees',
    'Mariners': 'Seattle Mariners',
    'Reds': 'Cincinnati Reds',
    'Blue Jays' : 'Toronto Blue Jays',
    'Mets' : 'New York Mets',
    'Phillies': 'Philadelphia Phillies',
    'Angels': 'Los Angeles Angels',
    'Braves': 'Atlanta Braves',
    'Cardinals': 'St. Louis Cardinals',
    'Tigers': 'Detroit Tigers',
    'Cubs': 'Chicago Cubs',
    'Indians': 'Cleveland Indians',
    'Guardians' : 'Cleveland Guardians',
    'Nationals': 'Washington Nationals',
    'Rockies': 'Colorado Rockies',
    'Marlins': 'Miami Marlins',
    'Royals': 'Kansas City Royals',
    'Twins': 'Minnesota Twins',
    'Pirates': 'Pittsburgh Pirates',
    'Rangers': 'Texas Rangers',
    'Orioles': 'Baltimore Orioles',
    'Diamondbacks': 'Arizona Diamondbacks',
    'D-backs' : 'Arizona Diamondbacks'
}

In [8]:
# Current runs scored
tables = pd.read_html('https://www.espn.com/mlb/stats/team')
runs_scored_table = pd.merge(tables[0], tables[1], left_on = tables[0].index, right_on = tables[1].index)
runs_scored_table.drop('key_0', axis = 1, inplace = True)
runs_scored_table = runs_scored_table[['Team', 'GP', 'R']]
runs_scored_table['Runs_162'] = runs_scored_table.R * (162.0/runs_scored_table.GP)

# Current runs allowed
tables = pd.read_html(f'https://www.foxsports.com/mlb/team-stats?category=pitching&season={dt.date.today().year}&seasonType=reg')
runs_allowed_table = tables[1]
runs_allowed_table = runs_allowed_table.iloc[:,[1,2,13]]
runs_allowed_table.columns = ['Team', 'Games', 'Runs_Allowed']
runs_allowed_table['Runs_Allowed_162'] = runs_allowed_table.Runs_Allowed * (162.0/runs_allowed_table.Games)
runs_allowed_table['Team'] = runs_allowed_table.Team.apply(lambda x: team_map[x])

/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
active_rosters = retrieve_all_active_rosters(file_name = None)
war = retrieve_current_year_WAR()
war[war.Name == 'Juan Soto']



====== WebDriver manager ======
Current google-chrome version is 104.0.5112
Get LATEST driver version for 104.0.5112
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/104.0.5112.79/chromedriver] found in cache


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=104.0.5112.79)
